In [1]:
# --utf-8---
import gzip
import pandas as pd
import sys
import io
import json
import re
import time
import ast
from zipfile import ZipFile
import json
import pickle
import os
from pathlib import Path
pd.set_option('max_colwidth',300)

In [4]:
python_files = sorted(Path("./processed_data/python/final/jsonl").glob('**/*.gz'))

In [5]:
for i in python_files:
    print(i)

processed_data/python/final/jsonl/test/python_test_0.jsonl.gz
processed_data/python/final/jsonl/train/python_train_0.jsonl.gz
processed_data/python/final/jsonl/train/python_train_1.jsonl.gz
processed_data/python/final/jsonl/train/python_train_10.jsonl.gz
processed_data/python/final/jsonl/train/python_train_11.jsonl.gz
processed_data/python/final/jsonl/train/python_train_12.jsonl.gz
processed_data/python/final/jsonl/train/python_train_13.jsonl.gz
processed_data/python/final/jsonl/train/python_train_2.jsonl.gz
processed_data/python/final/jsonl/train/python_train_3.jsonl.gz
processed_data/python/final/jsonl/train/python_train_4.jsonl.gz
processed_data/python/final/jsonl/train/python_train_5.jsonl.gz
processed_data/python/final/jsonl/train/python_train_6.jsonl.gz
processed_data/python/final/jsonl/train/python_train_7.jsonl.gz
processed_data/python/final/jsonl/train/python_train_8.jsonl.gz
processed_data/python/final/jsonl/train/python_train_9.jsonl.gz
processed_data/python/final/jsonl/vali

In [6]:
"""
Full List:
['repo', 'path', 'func_name', 'original_string', 'language', 'code', 
'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition']
"""

columns_long_list = ['repo', 'path', 'url', 'code',
                     'code_tokens', 'docstring', 'docstring_tokens', 
                     'language', 'partition']

columns_short_list = ["func_name",'code', 'docstring', 
                      'url', 'partition']

In [7]:
def jsonl_list_to_dataframe(file_list, columns=columns_long_list):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f, 
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

In [8]:
pydf = jsonl_list_to_dataframe(python_files, columns_short_list)

In [9]:
pydf.head(5)

,func_name,code,docstring,url,partition
0,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extracts video ID from URL.\n """"""\n return match1(url, r'youtu\.be/([^?/]+)') or \\n match1(url, r'youtube\.com/embed/([^/?]+)') or \\n match1(url, r'youtube\.com/v/([^/?]+)') or \\n match1(url, r'youtube\.com/watch/...",Extracts video ID from URL.,https://github.com/soimort/you-get/blob/b746ac01c9f39de94cac2d56f665285b0523b974/src/you_get/extractors/youtube.py#L135-L143,test
1,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""str->list\n Convert XML to URL List.\n From Biligrab.\n """"""\n rawurl = []\n dom = parseString(xml_data)\n for node in dom.getElementsByTagName('durl'):\n url = node.getElementsByTagName('url')[0]\n rawurl.append(url.chil...",str->list\n Convert XML to URL List.\n From Biligrab.,https://github.com/soimort/you-get/blob/b746ac01c9f39de94cac2d56f665285b0523b974/src/you_get/extractors/miomio.py#L41-L51,test
2,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37.atwikiimg.com/sitescript/pub/dksitescript/FC2.site.js\n Also com.hps.util.fc2.FC2EncrptUtil.makeMimiLocal\n L110""""""\n strSeed = ""gGddgPfeaf_gzyr""\n prehash = upid + ""_"" + strSeed\n return md5(prehash.encode('utf-8')).hexdigest()",From http://cdn37.atwikiimg.com/sitescript/pub/dksitescript/FC2.site.js\n Also com.hps.util.fc2.FC2EncrptUtil.makeMimiLocal\n L110,https://github.com/soimort/you-get/blob/b746ac01c9f39de94cac2d56f665285b0523b974/src/you_get/extractors/fc2video.py#L11-L17,test
3,fc2video_download,"def fc2video_download(url, output_dir = '.', merge = True, info_only = False, **kwargs):\n """"""wrapper""""""\n #'http://video.fc2.com/en/content/20151021bTVKnbEw'\n #'http://xiaojiadianvideo.asia/content/20151021bTVKnbEw'\n #'http://video.fc2.com/ja/content/20151021bTVKnbEw'\n #'http:...",wrapper,https://github.com/soimort/you-get/blob/b746ac01c9f39de94cac2d56f665285b0523b974/src/you_get/extractors/fc2video.py#L46-L57,test
4,dailymotion_download,"def dailymotion_download(url, output_dir='.', merge=True, info_only=False, **kwargs):\n """"""Downloads Dailymotion videos by URL.\n """"""\n\n html = get_content(rebuilt_url(url))\n info = json.loads(match1(html, r'qualities"":({.+?}),""'))\n title = match1(html, r'""video_title""\s*:\s*""(...",Downloads Dailymotion videos by URL.,https://github.com/soimort/you-get/blob/b746ac01c9f39de94cac2d56f665285b0523b974/src/you_get/extractors/dailymotion.py#L13-L35,test


In [10]:
def clean_up_comments(content):
    '''
    Collect Description and DocString
    '''
    content = re.sub(r"\s*#\s.*\n","\n", content)
    content =  re.sub(r"(\"{3}|\'{3})[\s\S]*(\"{3}|\'{3})[\n|\s]*","", content)
    content = re.sub(r"\n{2,}","\n",content)
    return content

In [11]:
pydf["code"] = pydf["code"].apply(clean_up_comments)

In [13]:
pydf[pydf["partition"]=="test"].to_csv("./processed_data/finetuning/test.csv", columns = ["docstring","code"])

In [14]:
pydf[pydf["partition"]=="train"].to_csv("./processed_data/finetuning/train.csv", columns = ["docstring","code"])

In [15]:
pydf[pydf["partition"]=="valid"].to_csv("./processed_data/finetuning/val.csv", columns = ["docstring","code"])

In [16]:
print(len(pydf[pydf["partition"]=="test"]))
print(len(pydf[pydf["partition"]=="train"]))
print(len(pydf[pydf["partition"]=="valid"]))

22176
412178
23107


In [271]:
# print(T5_config)